In [14]:
import geopandas as gpd
shapefile = gpd.read_file("public/Grid_5km_ENG_ONLY_with_PUBLIC_USE_dataset.shp")
shapefile.head()

ImportError: dlopen(/Users/gizatm/Applications/anaconda3/lib/python3.6/site-packages/fiona/ogrext.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libpoppler.71.dylib
  Referenced from: /Users/gizatm/Applications/anaconda3/lib/libgdal.20.dylib
  Reason: image not found

In [7]:
shapefile.plot(column='M17', cmap=None)

NameError: name 'shapefile' is not defined

In [ ]:
shapefile.crs

In [ ]:
shapefile['geometry']

In [ ]:
crs_shapefile = shapefile.copy()
crs_shapefile = crs_shapefile.to_crs(epsg=4326)

In [ ]:
crs_shapefile['geometry']

In [ ]:
for i in len(crs_shapefile['geometry']):
    print(i)

In [ ]:
crs_shapefile.index

In [ ]:
crs_shapefile.columns

In [ ]:
print(crs_shapefile['geometry'][0])

In [ ]:
crs_shapefile['geometry'][0]

In [ ]:
print(crs_shapefile['geometry'][0].centroid)

In [ ]:
master_shp = crs_shapefile.copy()

points = []

for index, row in crs_shapefile.iterrows():
    points.append(row['geometry'].centroid)

master_shp['geometry'] = points
master_shp.plot(column='A1', cmap=None)

In [ ]:
crs_shapefile.plot(column='A1', cmap=None)

In [ ]:
master_shp['geometry']

In [ ]:
# Print oats
master_shp.plot(column='A4', cmap=None)

In [ ]:
master_shp.to_csv('example.csv')

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

industrial_objects = gpd.read_file("industry.csv")

df = pd.DataFrame({
    'Location':industrial_objects['Name'],
    'Latitude':industrial_objects['Latitude'],
    'Longitude':industrial_objects['Longitude']
})

df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

df['Coordinates'] = list(zip(df.Longitude, df.Latitude))

df['Coordinates'] = df['Coordinates'].apply(Point)

gdf = gpd.GeoDataFrame(df, geometry='Coordinates')

fig, ax = plt.subplots()

ax.set_aspect('equal')

# Print total wheat
master_shp.plot(ax=ax, column='A1', cmap=None)

gdf.plot(ax=ax, color='red', markersize=5)

In [ ]:
master_shp.head()

In [ ]:
# Print total cattle
master_shp.plot(column='K299', cmap=None)

In [ ]:
# Print total poultry
master_shp.plot(column='N98', cmap=None)

In [ ]:
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt

df = pd.DataFrame(
    {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
     'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
     'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
     'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})

df['Coordinates'] = list(zip(df.Longitude, df.Latitude))

df['Coordinates'] = df['Coordinates'].apply(Point)

gdf = geopandas.GeoDataFrame(df, geometry='Coordinates')

print(gdf.head())

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# We restrict to South America.
ax = world[world.continent == 'South America'].plot(
    color='white', edgecolor='black')

# We can now plot our GeoDataFrame.
gdf.plot(ax=ax, color='red')

plt.show()

In [ ]:
import os
import pandas as pd
import pymongo
import json
import urllib.parse

class ads_mongo:
    dbname = ''
    addr = ''
    client = ''
    cli_db = ''

    def __init__(self, addr):
        self.addr = urllib.parse.quote_plus(addr)

    def connMongo(self, id, pwd, dbname):
        tmp_id = urllib.parse.quote_plus(id)
        tmp_pwd = urllib.parse.quote_plus(pwd)
        self.dbname = urllib.parse.quote_plus(dbname)
        self.client = pymongo.MongoClient('mongodb://%s:%s@%s/%s' % (tmp_id, tmp_pwd, self.addr, self.dbname))
        self.cli_db = self.client[self.dbname]
        print("Connected")
        return self.cli_db

    def uploadData(self, collection, filepath):
        db_col = self.cli_db[collection]
        cdir = os.path.abspath("")
        file_res = os.path.join(cdir, filepath)

        data = pd.read_csv(file_res)
        data_json = json.loads(data.to_json(orient='records'))
        db_col.remove()
        print("Uploading data to Database ... ")
        db_col.insert(data_json)
        print("Uploaded.")

    def downloadData(self, collection, filepath, format, query = {}):
        if format == 'json':
            print("Not support yet")

        elif format == 'csv':
            print("Downloading data to local ... ")
            cursor = self.cli_db[collection].find(query)
            df = pd.DataFrame(list(cursor))
            del df['_id']
            df.to_csv(filepath, index=False)
            print("Downloaded as csv format")

        else:
            print("No exist format")

    def collectionQuery(self, collection, operator, query = {}, save_in_local = False, filepath = 'result.csv'):
        op = operator
        if op == 'find':
            cs = self.cli_db[collection].find(query)
        elif op == 'aggregate':
            cs = self.cli_db[collection].aggregate(query)
        elif op == 'insert':
            self.cli_db[collection].insert(query)
            return
        elif op == 'delete':
            self.cli_db[collection].delete(query)
            return
        elif op == 'drop':
            self.cli_db[collection].drop()
            return
        else:
            print("Not exist operator")

        df = pd.DataFrame(list(cs))
        del df['_id']

        if save_in_local == True:
            df.to_csv(filepath, index=False)

        return df

In [ ]:
import os, sys
os_path = os.path.abspath("")

my_ads = ads_mongo('184.73.62.140')
client = my_ads.connMongo('user1', 'P@ssword', 'env')
file = "/Users/gizatm/Workshop/Python/geo/master.csv"
my_ads.uploadData('FieldShape', file)

In [ ]:
cs = client.FieldShape.find({ "ID": 500131})
df = pd.DataFrame(list(cs))
df.head()

--------------------------------

In [8]:
import pandas
import json

data_frame = pandas.read_csv(
    'master.csv'
)

data_frame[data_frame.columns[:]].head()

,Unnamed: 0,ID,E,N,ID_5kmsq,OID_,UNIQUE,A1,A2,A3,...,M9,M13M14,M17,M98,L1TOL5,L9L14,L98,N98,P90,geometry
0,0,500131,87500,7500,5.001319e+10,0,0.000000e+00,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,POINT (-6.352077147598139 49.8859481769574)
1,1,500132,92500,7500,5.001329e+10,0,0.000000e+00,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,POINT (-6.28267254272897 49.8885373817449)
2,2,500261,87500,12500,5.002619e+10,0,0.000000e+00,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,POINT (-6.356121648555834 49.93078789271393)
3,3,500262,92500,12500,5.002629e+10,0,0.000000e+00,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,POINT (-6.286653033960608 49.93338118711554)
4,4,500277,167500,12500,5.002772e+11,763,5.002772e+11,2.04042,0.48581,13.67248,...,2.09074,4.23281,171.97077,279.72148,1.06781,20.00601,21.07383,23.59964,4.97716,POINT (-5.243511148962318 49.96729097961222)


In [9]:
data_frame.dtypes

Unnamed: 0      int64
ID              int64
E               int64
N               int64
ID_5kmsq      float64
OID_            int64
UNIQUE        float64
A1            float64
A2            float64
A3            float64
A2A3          float64
A4            float64
A10A11        float64
A12           float64
A19           float64
A21           float64
A22           float64
A23           float64
A24           float64
A25           float64
A27           float64
A31           float64
A32           float64
A99G1         float64
B5B14         float64
B21           float64
B99           float64
C1            float64
G1            float64
G2            float64
G5            float64
K201          float64
K202          float64
K203          float64
K204          float64
K205          float64
K206          float64
K207          float64
K208          float64
K209          float64
K210          float64
K211          float64
K299          float64
M1M4M7        float64
M1            float64
M4        

In [10]:
json_result_string = data_frame.to_json(
    orient='records', 
    double_precision=12,
    date_format='iso'
)
json_result = json.loads(json_result_string)

In [ ]:
geojson = {
    'type': 'map',
    'grids': []
}
for record in json_result:
    geojson['grids'].append({
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [record['lng'], record['lat']],
        },
        'properties': record,
    })